# gyst (get your sheets together!) : energy and fluxes

This notebook is part of gyst (get your sheets together!).

Here, we get the energy, accretion rate, and magnetic fluxes from simulation data.

## 1) Setup

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import argparse
from scipy import interpolate
import tqdm
import nmmn.lsd, nmmn.misc
import os.path
import pandas as pd
import astropy.visualization

### 1.1) Get physical constants, definitions, and units

In [3]:
# many are related to grmonty, but we can set a black hole mass if we want to have a "feeling" for the physical values

global MP, ME, CL, GNEWT, KBOL, SIGMA_THOMSON, MSUN, LSUN, YEAR, MBH
global TPTE_DISK, TPTE_JET, THETAE_MAX
global M_unit, L_unit, T_unit, RHO_unit, U_unit, B_unit, Ne_unit

# all constants in cgs units
ME = 9.1093826e-28 # electron mass
MP = 1.67262171e-24 # proton mass
CL = 2.99792458e10 # speed of light
GNEWT = 6.6742e-8 # gravitational constant
KBOL = 1.3806505e-16 # Boltzmann constant
SIGMA_THOMSON = 0.665245873e-24 # Thomson cross-section
MSUN = 1.989e33 # solar mass
LSUN = 3.827e33 # solar luminosity
YEAR = 31536000 # seconds in a year

# temperature and beta-prescription (Mościbrodzka 2016)
TPTE_DISK = 20. # R_high
TPTE_JET = 1. # R_low
THETAE_MAX = 1000.
TP_OVER_TE = 100.0

sgra=1
m87=0
blazar=0
# grmonty units and BH mass
if (sgra):
    MBH = 4.5e6 * MSUN
    M_unit = 1.0e19
elif (m87):
    MBH = 6.2e9 * MSUN
    M_unit = 1.0e29

L_unit = GNEWT * MBH / (CL * CL)
T_unit = L_unit / CL
RHO_unit = M_unit / (L_unit*L_unit*L_unit)
U_unit = RHO_unit * CL * CL
B_unit = CL * np.sqrt(4. * np.pi * RHO_unit)
Ne_unit = RHO_unit / (MP + ME)

#MBH = 5.0e9 * MSUN
#M_unit = 2.0*10e11

In [5]:
%run -i harm_script2.py

# 1) Get fluxes and energy from a single simulation

### 1.1a) Get fluxes and save them to file

If we don't have the fluxes, we have to run this first to obtain them.

In [ ]:
ts = []
fs = []
md = []
#jem = []
#jtot = []

starti,endi=0,600
for dumpno in range(starti,endi+1):
    rd("dump%03d" % dumpno);

    rhor = 1+(1-a**2)**0.5
    ihor = iofr(rhor)
    #cvel()
    Tcalcud()
    ts.append(int(t)) #time
    fs.append(horfluxcalc(ihor)) #horizon flux
    md.append(mdotcalc(ihor)) #mass accretion rate
    #jem.append(jetpowcalc(0)[ihor]) #jet power (EM)
    #jtot.append(jetpowcalc(2)[ihor]) #jet power (total)
    
#write to file
fluxes_file = open("fluxes.dat","w+")
for line in range(len(ts)):
    fluxes_file.write(str(ts[line]))
    fluxes_file.write("\t")
    fluxes_file.write(str(fs[line]))
    fluxes_file.write("\t")
    fluxes_file.write(str(md[line]))
    fluxes_file.write("\n")
fluxes_file.close()

### 1.1b) Read fluxes from file

Alternatively, if we already have the fluxes, we can read them directly from a file.

In [ ]:
myfilename = "/work/gustavo/gyst/Be105_smallSANE/fluxes.dat"
myfile = open(myfilename, "r")
mydata = pd.read_csv(myfilename, header=None, sep='\t')

ts = mydata[0]
fs = mydata[1]
md = mydata[2]

### 1.2) Transform fluxes into arrays and get the correct units

In [ ]:
fs = np.array(fs)
fs_units = fs*B_unit*L_unit*L_unit
md = np.array(md)
md_units = md*M_unit/T_unit
phiflux = fs/np.sqrt(md)
phiflux = np.array(phiflux)

### 1.3) Plot fluxes

In [ ]:
fontsize=13
latexsize=18
myfigname = "fluxes_2D_MAD52.png"

hfont = {'fontname':'Helvetica'}
fig = plt.figure(figsize=[6.4, 8.4])
#fig = plt.figure(figsize=[12.8, 14.4])
gs = GridSpec(3,1)

ax1 = plt.subplot(gs[0,0])
ax1.plot(ts,fs_units*10**-25,color="black")
_, max_ = plt.ylim()
#plt.xlim(2000,7000)
plt.xticks([0, 1000, 2000,3000, 4000, 5000, 6000],fontsize=fontsize)
#plt.yticks([0,0.5,1.0,1.5,2.0],fontsize=fontsize)
plt.yticks([0,2.5,5.0,7.5,10.0],fontsize=fontsize)
plt.setp(ax1.get_xticklabels(), visible=True)
#plt.xlabel(r"$t\;(r_g/c)$")
#plt.ylabel(r"$\Phi_\mathrm{BH}\;(10^{27}\;\mathrm{G}\;\mathrm{cm}^{-2})$", fontsize=latexsize)
plt.ylabel(r"$\Phi_\mathrm{BH}\;(10^{25}\;\mathrm{G}\;\mathrm{cm}^{-2})$", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5)

ax2 = plt.subplot(gs[1,0], sharex=ax1)
ax2.plot(ts,md_units*10**10/MSUN*YEAR,color="black")
_, max_ = plt.ylim()
#plt.xlim(2000,7000)
plt.xticks([0, 1000, 2000,3000, 4000, 5000, 6000],fontsize=fontsize)
#plt.yticks([0,2,4,6,8],fontsize=fontsize)
plt.yticks([0,15,30,45,60],fontsize=fontsize)
#plt.xlabel(r"$t\;(r_g/c)$")
#plt.ylabel(r"$\dot M_\mathrm{acc}\;(10^{-7}M_\odot\;\mathrm{yr}^{-1})$", fontsize=latexsize)
plt.ylabel(r"$\dot M_\mathrm{acc}\;(10^{-10}M_\odot\;\mathrm{yr}^{-1})$", fontsize=latexsize)
plt.setp(ax2.get_xticklabels(), visible=True)
plt.grid(axis='x', alpha=0.5)

ax3 = plt.subplot(gs[2,0], sharex=ax1)
ax3.plot(ts,phiflux,color="black")
_, max_ = plt.ylim()
plt.xlim(0,6000)
plt.xticks([0, 1000, 2000,3000, 4000, 5000, 6000],fontsize=fontsize)
#plt.yticks([0,5,10,15,20,25],fontsize=fontsize)
#plt.yticks([0,0.3,0.6,0.9,1.2,1.5],fontsize=fontsize)
plt.xlabel(r"$t\;(r_g/c)$")
plt.ylabel(r"$\Phi_\mathrm{BH}/\sqrt{\dot M_\mathrm{acc}}$", fontsize=latexsize)
plt.grid(axis='x', alpha=0.5)

plt.tight_layout()

savef = 1
if (savef):
    plt.savefig(myfigname, dpi=150)

### 1.4) Get energy

### 1.5) Plot energy

Note: get energies by running the script getenergy.py

In [ ]:
time = list(range(225, 941))
time = np.array(time)
time=10*time
myfigname = "magenergy_MAD52.png"
savef = 0

fontsize=13
latexsize=18

myfilename_d = "/work/gustavo/gyst/Be105/magenergy_2D_disc.dat"
mylabel_d = "disc"
mycolor_d = "blue"
myfilename_o = "/work/gustavo/gyst/Be105/magenergy_2D_outflows.dat"
mylabel_o = "jet"
mycolor_o = "red"
myfilename_j = "/work/gustavo/gyst/Be105/magenergy_2D_jet.dat"

myfile_d = open(myfilename_d, "r")
mydata_d = pd.read_csv(myfilename_d, header=None)
magenext_d = mydata_d.values
magenext_d = np.array(magenext_d)
myfile_d.close()

myfile_o = open(myfilename_o, "r")
mydata_o = pd.read_csv(myfilename_o, header=None)
magenext_o = mydata_o.values
magenext_o = np.array(magenext_o)
myfile_o.close()

myfile_j = open(myfilename_j, "r")
mydata_j = pd.read_csv(myfilename_j, header=None)
magenext_j = mydata_j.values
magenext_j = np.array(magenext_j)
myfile_j.close()

hfont = {'fontname':'Helvetica'}
fig = plt.figure(figsize=[6.4, 9.6])
#fig = plt.figure(figsize=[12.8, 14.4])
gs = GridSpec(3,1)

ax1 = plt.subplot(gs[0,0])
ax1.plot(time,magenext_d*B_unit*B_unit*L_unit*L_unit*L_unit, color=mycolor_d, label=mylabel_d)
#ax1.plot(time,np.log10(magenext_d*B_unit*B_unit*L_unit*L_unit*L_unit), color=mycolor_d, label=mylabel_d)
ax1.set_yscale('log')
#_, max_ = plt.ylim()
#plt.ylim(0,4)
#plt.xlabel(r"$t\;(10r_g/c)$", fontsize=16)
plt.ylabel(r"$\mathrm{Magnetic\;energy}\;(\mathrm{erg})$", fontsize=latexsize)
#plt.ylabel(r"$B^2/8\pi\;(10^{38}\;\mathrm{erg})$", fontsize=18)
plt.xticks([2000,3000,4000,5000,6000,7000,8000,9000,10000],fontsize=fontsize)
plt.setp(ax1.get_xticklabels(), visible=True)
plt.yticks([10**41,10**43,10**45,10**47,10**49,10**51],fontsize=fontsize)
plt.grid(axis='x', alpha=0.5)
plt.legend(loc='lower right')

ax2 = plt.subplot(gs[1,0], sharex=ax1)
ax2.plot(time,magenext_o*B_unit*B_unit*L_unit*L_unit*L_unit, color=mycolor_o, label='outflows')
#ax2.plot(time,np.log10(magenext_j*B_unit*B_unit*L_unit*L_unit*L_unit), color=mycolor_j, label=mylabel_j)
ax2.set_yscale('log')
#_, max_ = plt.ylim()
#plt.ylim(0,4)
plt.xlim(2000,9650)
#plt.xlabel(r"$t\;(r_g/c)$", fontsize=latexsize)
plt.ylabel(r"$\mathrm{Magnetic\;energy}\;(\mathrm{erg})$", fontsize=latexsize)
#plt.ylabel(r"$B^2/8\pi\;(10^{38}\;\mathrm{erg})$", fontsize=18)
plt.xticks([2000,3000,4000,5000,6000,7000,8000,9000,10000],fontsize=fontsize)
plt.yticks([10**41,10**43,10**45,10**47,10**49,10**51],fontsize=fontsize)
plt.legend(loc='lower right')
plt.grid(axis='x', alpha=0.5)
plt.tight_layout()

ax3 = plt.subplot(gs[2,0], sharex=ax1)
ax3.plot(time,magenext_j*B_unit*B_unit*L_unit*L_unit*L_unit, color='green', label=mylabel_j)
#ax2.plot(time,np.log10(magenext_j*B_unit*B_unit*L_unit*L_unit*L_unit), color=mycolor_j, label=mylabel_j)
ax3.set_yscale('log')
#_, max_ = plt.ylim()
#plt.ylim(0,4)
plt.xlim(2000,9650)
plt.xlabel(r"$t\;(r_g/c)$", fontsize=latexsize)
plt.ylabel(r"$\mathrm{Magnetic\;energy}\;(\mathrm{erg})$", fontsize=latexsize)
#plt.ylabel(r"$B^2/8\pi\;(10^{38}\;\mathrm{erg})$", fontsize=18)
plt.xticks([2000,3000,4000,5000,6000,7000,8000,9000,10000],fontsize=fontsize)
plt.yticks([10**41,10**43,10**45,10**47,10**49,10**51],fontsize=fontsize)
plt.legend(loc='lower right')
plt.grid(axis='x', alpha=0.5)
plt.tight_layout()

savef=0
if (savef):
    plt.savefig(myfigname, dpi=150)

#### 1.5.1) Print mean and maximum energies

In [ ]:
print("mean energy, disc = "+str(np.mean(magenext_d*B_unit*B_unit*L_unit*L_unit*L_unit)))
print("mean energy, outflows = "+str(np.mean(magenext_o*B_unit*B_unit*L_unit*L_unit*L_unit)))
print("mean energy, jet = "+str(np.mean(magenext_j*B_unit*B_unit*L_unit*L_unit*L_unit)))
print("max energy, disc = "+str(np.max(magenext_d*B_unit*B_unit*L_unit*L_unit*L_unit)))
print("max energy, outflows = "+str(np.max(magenext_o*B_unit*B_unit*L_unit*L_unit*L_unit)))
print("max energy, jet = "+str(np.max(magenext_j*B_unit*B_unit*L_unit*L_unit*L_unit)))

## 2) Compare MAD and SANE fluxes

Assuming we already have fluxes from two simulations (i.e. one MAD and one SANE), we can compare them by plotting both on the same figure.

In [ ]:
myfilename_MAD = "/work/gustavo/gyst/Be105_smallMAD/fluxes.dat"
myfile_MAD = open(myfilename_MAD, "r")
mydata_MAD = pd.read_csv(myfilename_MAD, header=None, sep='\t')

ts_MAD = mydata_MAD[0]
fs_MAD = mydata_MAD[1]
md_MAD = mydata_MAD[2]

fs_MAD = np.array(fs_MAD)
fs_MAD_units = fs_MAD*B_unit*L_unit*L_unit
md_MAD = np.array(md_MAD)
md_MAD_units = md_MAD*M_unit/T_unit
phiflux_MAD = fs_MAD/np.sqrt(md_MAD)
phiflux_MAD = np.array(phiflux_MAD)

myfilename_SANE = "/work/gustavo/gyst/Be105_smallSANE/fluxes.dat"
myfile_SANE = open(myfilename_SANE, "r")
mydata_SANE = pd.read_csv(myfilename_SANE, header=None, sep='\t')

ts_SANE = mydata_SANE[0]
fs_SANE = mydata_SANE[1]
md_SANE = mydata_SANE[2]

fs_SANE = np.array(fs_SANE)
fs_SANE_units = fs_SANE*B_unit*L_unit*L_unit
md_SANE = np.array(md_SANE)
md_SANE_units = md_SANE*M_unit/T_unit
phiflux_SANE = fs_SANE/np.sqrt(md_SANE)
phiflux_SANE = np.array(phiflux_SANE)

fontsize=13
latexsize=18
color_MAD = "red"
color_SANE = "blue"
myfigname = "fluxes_2D.png"

hfont = {'fontname':'Helvetica'}
fig = plt.figure(figsize=[6.4, 8.4])
#fig = plt.figure(figsize=[12.8, 14.4])
gs = GridSpec(3,1)

ax1 = plt.subplot(gs[0,0])
ax1.plot(ts_MAD,fs_MAD_units*10**-25,color=color_MAD,label="MAD")
ax1.plot(ts_SANE,fs_SANE_units*10**-25,color=color_SANE,label="SANE")
_, max_ = plt.ylim()
#plt.xlim(2000,7000)
plt.xticks([0, 1000, 2000,3000, 4000, 5000, 6000],fontsize=fontsize)
#plt.yticks([0,0.5,1.0,1.5,2.0],fontsize=fontsize)
plt.yticks([0,50,100],fontsize=fontsize)
plt.setp(ax1.get_xticklabels(), visible=True)
#plt.xlabel(r"$t\;(r_g/c)$")
#plt.ylabel(r"$\Phi_\mathrm{BH}\;(10^{27}\;\mathrm{G}\;\mathrm{cm}^{-2})$", fontsize=latexsize)
plt.ylabel(r"$\Phi_\mathrm{BH}\;(10^{25}\;\mathrm{G}\;\mathrm{cm}^{-2})$", fontsize=latexsize)
plt.legend()
plt.grid(axis='x', alpha=0.5)

ax2 = plt.subplot(gs[1,0], sharex=ax1)
ax2.plot(ts_MAD,md_MAD_units*10**10/MSUN*YEAR,color=color_MAD,label="MAD")
ax2.plot(ts_SANE,md_SANE_units*10**10/MSUN*YEAR,color=color_SANE,label="SANE")
_, max_ = plt.ylim()
#plt.xlim(2000,7000)
plt.xticks([0, 1000, 2000,3000, 4000, 5000, 6000],fontsize=fontsize)
#plt.yticks([0,2,4,6,8],fontsize=fontsize)
plt.yticks([0,100,200],fontsize=fontsize)
#plt.xlabel(r"$t\;(r_g/c)$")
#plt.ylabel(r"$\dot M_\mathrm{acc}\;(10^{-7}M_\odot\;\mathrm{yr}^{-1})$", fontsize=latexsize)
plt.ylabel(r"$\dot M_\mathrm{acc}\;(10^{-10}M_\odot\;\mathrm{yr}^{-1})$", fontsize=latexsize)
plt.setp(ax2.get_xticklabels(), visible=True)
plt.legend()
plt.grid(axis='x', alpha=0.5)

ax3 = plt.subplot(gs[2,0], sharex=ax1)
ax3.plot(ts_MAD,phiflux_MAD,color=color_MAD,label="MAD")
ax3.plot(ts_SANE,phiflux_SANE,color=color_SANE,label="SANE")
_, max_ = plt.ylim()
plt.xlim(0,6000)
plt.xticks([0, 1000, 2000,3000, 4000, 5000, 6000],fontsize=fontsize)
#plt.yticks([0,5,10,15,20,25],fontsize=fontsize)
plt.yticks([0,10,20,30],fontsize=fontsize)
plt.xlabel(r"$t\;(r_g/c)$")
plt.ylabel(r"$\Phi_\mathrm{BH}/\sqrt{\dot M_\mathrm{acc}}$", fontsize=latexsize)
plt.legend()
plt.grid(axis='x', alpha=0.5)

plt.tight_layout()

savef = 1
if (savef):
    plt.savefig(myfigname, dpi=150)

In [ ]:
time = list(range(0, 601))
time = np.array(time)
time=10*time
myfigname = "magenergy_sgra.png"
savef = 0

fontsize=13
latexsize=18

mylabel_MAD = "MAD"
mylabel_SANE = "SANE"
mycolor_MAD = "red"
mycolor_SANE = "blue"

myfilename_d_MAD = "/work/gustavo/gyst/Be105_smallMAD/magenergy_2D_disc.dat"
myfilename_j_MAD = "/work/gustavo/gyst/Be105_smallMAD/magenergy_2D_jet.dat"

myfilename_d_SANE = "/work/gustavo/gyst/Be105_smallSANE/magenergy_2D_disc.dat"
myfilename_j_SANE = "/work/gustavo/gyst/Be105_smallSANE/magenergy_2D_jet.dat"

myfile_d_MAD = open(myfilename_d_MAD, "r")
mydata_d_MAD = pd.read_csv(myfilename_d_MAD, header=None)
magenext_d_MAD = mydata_d_MAD.values
magenext_d_MAD = np.array(magenext_d_MAD)
myfile_d_MAD.close()

myfile_d_SANE = open(myfilename_d_SANE, "r")
mydata_d_SANE = pd.read_csv(myfilename_d_SANE, header=None)
magenext_d_SANE = mydata_d_SANE.values
magenext_d_SANE = np.array(magenext_d_SANE)
myfile_d_SANE.close()

myfile_j_MAD = open(myfilename_j_MAD, "r")
mydata_j_MAD = pd.read_csv(myfilename_j_MAD, header=None)
magenext_j_MAD = mydata_j_MAD.values
magenext_j_MAD = np.array(magenext_j_MAD)
myfile_j_MAD.close()

myfile_j_SANE = open(myfilename_j_SANE, "r")
mydata_j_SANE = pd.read_csv(myfilename_j_SANE, header=None)
magenext_j_SANE = mydata_j_SANE.values
magenext_j_SANE = np.array(magenext_j_SANE)
myfile_j_SANE.close()

hfont = {'fontname':'Helvetica'}
fig = plt.figure(figsize=[6.4, 9.6])
#fig = plt.figure(figsize=[12.8, 14.4])
gs = GridSpec(2,1)

ax1 = plt.subplot(gs[0,0])
ax1.title("Disc")
ax1.plot(time,magenext_d_MAD*B_unit*B_unit*L_unit*L_unit*L_unit, color=mycolor_MAD, label=mylabel_MAD)
ax1.plot(time,magenext_d_SANE*B_unit*B_unit*L_unit*L_unit*L_unit, color=mycolor_SANE, label=mylabel_SANE)
#ax1.plot(time,np.log10(magenext_d*B_unit*B_unit*L_unit*L_unit*L_unit), color=mycolor_d, label=mylabel_d)
ax1.set_yscale('log')
#_, max_ = plt.ylim()
plt.ylim(10**33,10**41)
#plt.xlabel(r"$t\;(10r_g/c)$", fontsize=16)
plt.ylabel(r"$\mathrm{Magnetic\;energy}\;(\mathrm{erg})$", fontsize=latexsize)
#plt.ylabel(r"$B^2/8\pi\;(10^{38}\;\mathrm{erg})$", fontsize=18)
plt.xticks([0,1000,2000,3000,4000,5000,6000],fontsize=fontsize)
plt.setp(ax1.get_xticklabels(), visible=True)
plt.yticks([10**33,10**35,10**37,10**39,10**41],fontsize=fontsize)
plt.grid(axis='x', alpha=0.5)
plt.legend(loc='lower right')

ax2 = plt.subplot(gs[1,0], sharex=ax1)
ax1.title("Jet")
ax2.plot(time,magenext_j_MAD*B_unit*B_unit*L_unit*L_unit*L_unit, color=mycolor_MAD, label=mylabel_MAD)
ax2.plot(time,magenext_j_SANE*B_unit*B_unit*L_unit*L_unit*L_unit, color=mycolor_SANE, label=mylabel_SANE)
#ax2.plot(time,np.log10(magenext_j*B_unit*B_unit*L_unit*L_unit*L_unit), color=mycolor_j, label=mylabel_j)
ax2.set_yscale('log')
#_, max_ = plt.ylim()
plt.ylim(10**33,10**41)
plt.xlim(0,6000)
#plt.xlabel(r"$t\;(r_g/c)$", fontsize=latexsize)
plt.ylabel(r"$\mathrm{Magnetic\;energy}\;(\mathrm{erg})$", fontsize=latexsize)
#plt.ylabel(r"$B^2/8\pi\;(10^{38}\;\mathrm{erg})$", fontsize=18)
plt.xticks([0,1000,2000,3000,4000,5000,6000],fontsize=fontsize)
plt.yticks([10**33,10**35,10**37,10**39,10**41],fontsize=fontsize)
plt.legend(loc='lower right')
plt.grid(axis='x', alpha=0.5)
plt.tight_layout()

savef=0
if (savef):
    plt.savefig(myfigname, dpi=150)